In [1]:
import pandas as pd

# CSV 파일 내용을 확인
file_path = '../data/justwatch.csv'
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.readlines()

# 출력할 줄 수를 제한하여 확인 (예: 처음 10줄)
print('\n'.join(content[:10]))

/title/original_title/year/season_episode/runtime/genre/age_rating/Production country

0/Ojingeo Geim/Squid Game/2021/2/55min/액션, 드라마, 스릴러/19/대한민국

1/12인의 심판자/The Twelve/2019/2/54min/드라마, 스릴러//벨기에

2/희생자 게임/誰是被害者/2020/2/1시간 0분/드라마, 범죄, 스릴러/19/대만

3/스위트 투스: 사슴뿔을 가진 소년/Sweet Tooth/2021/3/52min/SF, 드라마, 판타지, 액션/15/미국

4/Sweet Home//2020/3/1시간 0분/스릴러, 드라마, 판타지, 공포, SF, 범죄//대한민국

5/외교관/The Diplomat/2023/2/48min/드라마, 전쟁, 스릴러, 코미디/15/미국

6/굿 닥터/The Good Doctor/2017/7/43min/드라마/15/미국

7/워킹 데드/The Walking Dead/2010/11/46min/드라마, SF, 액션, 공포, 스릴러/19/미국

8/비르기트: 왕국, 권력, 영광/Borgen - Riget, Magten og Æren/2022/4/58min/전쟁, 드라마, Made in Europe/15/덴마크



In [2]:
# CSV 파일 로드 (에러 있는 줄 건너뛰기)
df = pd.read_csv('../data/justwatch.csv', sep="/", encoding='utf-8', index_col=0, on_bad_lines='skip')

# CSV 파일의 열 이름 확인
print(df.columns)


Index(['title', 'original_title', 'year', 'season_episode', 'runtime', 'genre',
       'age_rating', 'Production country'],
      dtype='object')


In [3]:
# # CSV 파일에서 제목을 추출 (에러 있는 줄 건너뛰기)
# df = pd.read_csv('../data/justwatch.csv', on_bad_lines='skip')

# 제목 리스트 추출
titles = df['original_title'].tolist()
len(titles)

1090

In [27]:
# 오징어게임으로 해보는 테스트 페이지

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# 웹 드라이버 설정
driver = webdriver.Chrome()

# IMDB 검색 페이지로 이동
search_url = 'https://www.imdb.com/find/?q=squid%20game&ref_=nv_sr_sm'
driver.get(search_url)

try:
    # 첫 번째 검색 결과가 로드될 때까지 기다림
    first_result = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-8cf8f1-0.cGPWeC > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-ffc93fc1-2.ditJlF > ul > li:nth-child(1)'))
    )

    # 첫 번째 검색 결과 클릭
    first_result.click()

    # 페이지가 로드될 때까지 기다림
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'h1'))
    )

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 드라마 정보 추출
    title = soup.select_one('h1').text.strip() if soup.select_one('h1') else None
    rating = soup.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-5.cxlubq > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK').text.strip() if soup.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-5.cxlubq > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK') else None

    # 사용자 리뷰 점수 추출
    user_review_score = soup.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > ul > li:nth-child(1) > a > span > span.score')
    user_review_score = user_review_score.text.strip() if user_review_score else None

    # Cast & Crew 페이지로 이동
    cast_crew_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li:nth-child(1) > a'))
    )
    cast_crew_button.click()

    # Cast & Crew 페이지가 로드될 때까지 충분히 기다림
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'h1'))
    )

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 감독 정보 추출
    director = []
    directors = soup.select('#fullcredits_content > table:nth-child(2)')
    for d in directors:
        name = d.select_one('td.name a')
        if name:
            director.append(name.text.strip())
    
   # 배우 정보 추출 (5명까지만)
    actors = []
    cast_list = soup.select('table.cast_list tr')
    for tr in cast_list[1:10]:  # 첫 번째 tr은 헤더라서 건너뜀, 5명까지만 추출
        td = tr.find_all('td')
        if len(td) > 1:
            actor_name = td[1].text.strip()
            actors.append(actor_name)


    # 출력
    print("Title:", title)
    print("Rating:", rating)
    print("User Review Count:", user_review_score)
    print("Directors:", director)
    print("Actors:", actors)

except Exception as e:
    print("An error occurred:", e)

# 웹 드라이버 종료
driver.quit()


Title: 오징어 게임
Rating: 8.0
User Review Count: 2.6K
Directors: ['Hwang Dong-hyuk']
Actors: ['Lee Jung-jae', 'Park Hae-soo', 'Yasushi Iwaki', 'Hoyeon']


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

# 크롤링한 데이터를 저장할 리스트
dramas = []

# Selenium을 사용하여 웹 드라이버 설정
driver = webdriver.Chrome()

# IMDB에서 각 제목을 검색하여 데이터 크롤링
for title in titles[:5]:  # 여기서는 테스트를 위해 상위 5개 제목만 사용
    try:
        # IMDB 검색 페이지로 이동
        search_url = 'https://www.imdb.com/find?q=' + '+'.join(title.split())
        driver.get(search_url)

        # 첫 번째 검색 결과가 로드될 때까지 기다림
        first_result = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-8cf8f1-0.cGPWeC > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-ffc93fc1-2.ditJlF > ul > li:nth-child(1)'))
        )

        # 첫 번째 검색 결과 클릭
        first_result.click()

        # 드라마 페이지가 로드될 때까지 기다림
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h1'))
        )

        # 페이지 소스 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 드라마 정보 추출
        title = soup.select_one('h1').text.strip() if soup.select_one('h1') else None
        rating = soup.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-5.cxlubq > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK').text.strip() if soup.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-5.cxlubq > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK') else None
        

        # 사용자 리뷰 점수 추출
        user_review_score = soup.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > ul > li:nth-child(1) > a > span > span.score')
        user_review_score = user_review_score.text.strip() if user_review_score else None

        # Cast & Crew 페이지로 이동
        cast_crew_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li:nth-child(1) > a'))
        )
        cast_crew_button.click()

        # Cast & Crew 페이지가 로드될 때까지 충분히 기다림
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h1'))
        )

        # 페이지 소스 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 감독 정보 추출
        directors = []
        director_elements = soup.select('#fullcredits_content > table:nth-child(2) a')
        for d in director_elements:
            directors.append(d.text.strip())

        # 배우 정보 추출 (5명까지만)
        actors = []
        cast_list = soup.select('table.cast_list tr')
        for tr in cast_list[1:10]:  # 첫 번째 tr은 헤더라서 건너뜀, 5명까지만 추출
            td = tr.find_all('td')
            if len(td) > 1:
                actor_name = td[1].text.strip()
                actors.append(actor_name)

        drama = {
            'title': title,
            'rating': rating,
            'user_review_count': user_review_score,
            'directors': ', '.join(directors),
            'actors': ', '.join(actors)
        }
        dramas.append(drama)

    except Exception as e:
        print(f"Failed to get data for {title}: {e}")
        continue

# 드라이버 종료
driver.quit()


Failed to get data for nan: 'float' object has no attribute 'split'


In [39]:
# 데이터프레임으로 변환
dramas_df = pd.DataFrame(dramas)

# 데이터프레임 출력
dramas_df


,title,rating,user_review_count,directors,actors
0,오징어 게임,8.0,2.6K,Hwang Dong-hyuk,"Lee Jung-jae, Park Hae-soo, Yasushi Iwaki, Hoy..."
1,The Twelve,6.8,56,"Daniel Nettheim, Sian Davies, Greg McLean, Cat...","Sam Neill, Brooke Satchwell, Kate Mulvany, Dam..."
2,Shei shi bei hai zhe,None,None,"Kuan-Chung Chen, David Chuang","Wei-Ning Hsu, Hsiao-chuan Chang, Shih-Sian Wan..."
3,스위트 투스: 사슴뿔을 가진 소년,7.7,775,"Toa Fraser, Jim Mickle, Robyn Grace, Ciarán Fo...","Nonso Anozie, Christian Convery, Adeel Akhtar,..."


In [ ]:

# CSV 파일로 저장
dramas_df.to_csv('../data/netflix_dramas.csv', index=False)